<CENTER>
<H1 style="color:red">
SMCE Heliophysics DaskHub S3 Bucket Tutorial
</H1>
</CENTER>
<!--<img src="./banner.jpg">-->

# Notebook to process data in SMCE helio-public S3 bucket using Dask

<i>Note:  This document is maintained through the SMCE HSD Cloud gitlab.</i>

This is a simple example showing how to get a list of FITS files, run them through Dask workers to pull them from disk and examine the header keyword(s). 

You can use this notebook to test out various parameters you might feed to Dask; Consider the 'batch size', number of workers, number of cores per worker and memory per worker. Use the dashboard link to inspect how Dask is performing. Try both manual and automatic scaling strategies. See if you can get it to process 100 files in 20 sec or less!

## First:  What is S3? 

S3 stands for "Simple Storage Service," which provides object storage for for AWS.  https://aws.amazon.com/s3/ 

It allows people to query and access data from a common location reference.  The buckets can be made <a href="https://stackoverflow.com/q/16784052">web accessible to users outside of daskhub</a> if web access is enabled.    

S3 buckets are individual storage elements. 

## Accessing S3 buckets

To <a href= "https://awscli.amazonaws.com/v2/documentation/api/latest/reference/s3/ls.html">get a list of the S3 buckets</a> on the SMCE Daskhub, enter this at a terminal prompt : <br>
`aws s3 ls`

To view the contents of a specific bucket, reference it with s3:// <br>
`aws s3 ls s3://helio-public/`
>            PRE SDO/
>            PRE SOHO/

(Note: "PRE" stands for prefix, so SDO/ is an AWS prefix with name SDO.) 
<hr>

The external reference for this bucket is https://helio-public.s3.us-east-1.amazonaws.com/

## Basic commands using S3 buckets

To create a new directory, just reference it:<br>
`aws s3 ls s3://helio-public/yourname/yourdir`

then you can copy to the bucket as if it was a unix folder:<br>
`aws s3 cp yourfile s3://helio-public/yourname/yourdir`

copying multiple files is a bit more intricate, you need to put the multiple files in a directory first:<br>
`aws s3 cp sourcedir/ s3://helio-public/yourname/yourdir --recursive`

if you need access to a bucket that has restricted access, you have to run aws-mfa first:<br>
`~/aws-mfa default`

where default is a profile. To see available profiles:<br>
`cat ~/.aws/credentials`

you may need to change it to have execute permission first:<br>
`chmod 755 ~/aws-mfa`

## (The code below needs to be commented for instructional purposes)

In [1]:
import boto3
from botocore import UNSIGNED
from botocore.config import Config

import dask
import io
import logging
import s3fs

from astropy.io import fits
from dask.distributed import Client
from os import listdir
from os.path import isfile, join
from re import search

## 0. Configuration 

In [2]:
# name of the bucket to upload to
bucket_name = 'gov-nasa-hdrl-data1'

# location in the bucket to use (a days worth of 211 A data from AIA on SDO for the date 2022-11-27)
bucket_path = 'sdo/aia/20221127/0211/'

# number of workers to use, for automatic scaling, our max number
n_workers = 10

# memory per worker (in Gb)
w_memory = 2

# cores per worker
w_cores = 2

# number of files to test against (360 max)
n_files = 100

# Number of files we release to be worked on by all workers at a time
# the higher the number the more files being processed concurrently, but also
# the greater the memory consumed. 
batch_size = 50

# use Manual (if False, then uses Automatic scaling)
use_manual_scaling = False

## 1. Initialize the cluster and assign the client to the cluster, display the cluster widget

In [3]:
from dask_gateway import Gateway, GatewayCluster
gateway = Gateway()
options = gateway.cluster_options()

# We're setting some defaults here just for grins... 
# I like the pangeo/base-notebook image for the workers since it has almost every library you'd need on a worker
# In our environment, without setting these, the widget will default to the same image that the notebook itself is running, 
# as well as 2 cores and 4GB memory per worker

options.worker_cores=w_cores
options.worker_memory=w_memory
# options

In [4]:
cluster = gateway.new_cluster(options)
client = cluster.get_client()

if use_manual_scaling:
    
    # manual scaling (n_workers defined above)
    cluster.scale(n_workers)
    
else:
    
    # Adaptively scale between 1 and n_workers (the max)
    cluster.adapt(minimum=1, maximum=n_workers)

# uncomment this if you want to use the GUI
#cluster

In [5]:
# create client, show url we can go to to monitor progress
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/daskhub.0f89be2f89864308a352fbda0ca887d3/status,


## 2. Scan data from bucket and make a simple list of file names

In [6]:
# get our list of files/s3 objects
cached = True
import json
if (cached):
    with open ('s3_data.json') as f:
        s3_files = json.load(f)['files']

else:

    # initialize connection to S3 bucket
    #s3_client = boto3.resource('s3')
    s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
    objs = s3_client.list_objects(Bucket=bucket_name, Prefix=bucket_path)

    #print (objs)
    # Iterates through all the objects, doing the pagination for you. Each obj
    # is an ObjectSummary, so it doesn't contain the body. You'll need to call
    # get to get the whole body.
    s3_files = []
    for obj in objs['Contents']: #bucket.objects.all():
        #print (obj)
        key = obj['Key']

        if search ('fits', key):
            s3_files.append(key)
    
    # write / cache files to local listing (speed purposes)

    with open('s3_data.json', 'w') as outfile:
        json.dump({'files' : s3_files}, outfile)
    

s3_files[0]


'sdo/aia/20221127/0211/sdo_aia_h2_20221127T000000_0211_v1.fits'

## 3. Define some routines we will use for doing work with Dask

In [7]:
def open_fits_s3 (s3_file_name:str, bucket_name:str)->object:
    
    """ Open a FITS file on an S3 bucket, pass back a binary blob of file info.
    """
 
    fs = s3fs.S3FileSystem()
    
    with fs.open(bucket_name+'/'+s3_file_name, 'rb') as f:
        #fits_hdul = fits.open(io.BytesIO(f.read()))
        #fits_hdul.info()
    
        # return bytes
        return f.read()

    return None

def get_header(f_bytes:object)->str:
    
    """ Marshal the bytes into a astropy FITS object, and pass back 
    """
    return fits.open(io.BytesIO(f_bytes))   
    
def work_on_data (client:dask.distributed.client.Client, bucket_name:str, files:list=[])->int:
    
    """ 
    Main routine which Dask will use to 'do work'. Each worker will run this.
    """
    
    # pull files from S3 as bytes
    f_bytes_list = client.map(open_fits_s3, files, bucket_name=bucket_name)
    
    # read bytes as FITS file, grab header
    hduls = client.map(get_header, f_bytes_list)
    
    # trigger distributed task, marshall result back to local memory
    headers=[]
    try:
        headers = client.gather(hduls)
    except Exception as er:
        print (f"Caught exception {er}")
    
    # return the primary header back for analysis
    return [hdr[1].header for hdr in headers]


## 4. Do the cloud processing, using Dask to 'burst' into other VMs
Using our gathered list of FITS files, chunk it out in batches and provide file list chunks to the workers

In [8]:
%%time

if n_files > len(s3_files):
    n_files = len(s3_files)
    
def chunks(lst, n):
    """ program to divide our file list into chunks for each worker """
    n = max(1, n)
    return (lst[i:i+n] for i in range(0, len(lst), n))

print (f"workers: {n_workers}, cores/worker:{w_cores}, mem/worker: {w_memory}")

for files_to_process in chunks(s3_files[:n_files], batch_size):

    r = work_on_data(client, bucket_name, files_to_process)
    print (f"client:%s Finished %s files" % (client,len(r)))


workers: 10, cores/worker:2, mem/worker: 2
client:<Client: 'tls://192.168.27.39:8786' processes=5 threads=10, memory=10.00 GiB> Finished 50 files
client:<Client: 'tls://192.168.27.39:8786' processes=5 threads=10, memory=10.00 GiB> Finished 50 files
CPU times: user 2.78 s, sys: 1.56 s, total: 4.34 s
Wall time: 20.3 s


In [9]:
# take first result, and check for INSTRUME header keyword
print (r[0]['INSTRUME'])

AIA_2


In [10]:
print (r[0])

SIMPLE  =                    T / file does conform to FITS standard             BITPIX  =                   32 / data type of original image                    NAXIS   =                    2 / dimension of original image                    NAXIS1  =                 5000 / length of original image axis                  NAXIS2  =                 5000 / length of original image axis                  REQUESTI= 'NotFromExport'      / Export request id if this program was invoked bDATE    = '2022-12-20T19:06:41.000' / [ISO] Date_time of processing; ISO 8601   TELESCOP= 'SDO/AIA '           / For AIA: SDO/AIA                               INSTRUME= 'AIA_2   '           / For AIA: AIA_ATA1, AIA_ATA2, AIA_ATA3 or AIA_ATWAVELNTH=                  211 / [angstrom] Wavelength                          WAVEUNIT= 'angstrom'           / Wavelength unit: angstrom                      WAVE_STR= '211_THIN'           / Wavelength_FilterPosition                      CAMERA  =                    2 / For AIA